# Auditing for Spatial Fairness

This notebook runs the experiments. The methods are implemented in the `src/function.py` file.

In [1]:
import pandas as pd
import numpy as np
import folium


import sys
sys.path.append('./src')
                
from functions import *

# 1. Select the dataset

You can select one of the following:
- LAR (`data/LAR.csv`) contains the modified Loan/Application Register records in the US for Bank of America for the year 2021; the dataset is created by `data/LAR/create_LAR.ipynb`.
- Crime (`data/Crime.csv`) contains predictions about crime incidents in the city of Los Angeles from 2010–2019; the predictive model is a Random Forest Classifier and the dataset is created by `data/Crime/create_Crime.ipynb`.
- Synth_fair/Synth_unfair/Semisynth (`/data/Synth_fair.csv`, `/data/Synth_unfair.csv`, `/data/Semisynth.csv`) are synthetic datasets create by the notebooks in `data/Synth/`.

In [2]:
## load the LAR dataset
df = load_data('./data/LAR.csv')
label = 'action_taken'
## df = filterbbox(df, -87.634938, 24.523096, -80.031362, 31.000888) ## florida
## df = filterbbox(df, -80.8736, 25.13742, -80.06279, 25.979434) # miami


## load the CRIME_serious dataset
# df = load_data('./data/Crime.csv')
# label = 'pred'


# load a synthetic dataset
# df = load_data('./data/Synth_fair.csv') ## FAIR
# label = 'label'


# df = load_data('./data/Synth_unfair.csv') ## UNFAIR
# label = 'label'


# df = load_data('./data/Semisynth.csv') ## FAIR
# label = 'label'


N, P = get_stats(df, label)

print(f'N={N} points')
print(f'P={P} positives')
df.head()

N=206418 points
P=127286 positives


,action_taken,census_tract,location,lat,lon
0,1,10003015200,"(39.7070504, -75.5832416)",39.707050,-75.583242
1,1,6059086502,"(33.8503559, -117.9121351)",33.850356,-117.912135
2,1,26163596100,"(42.1014482, -83.1602786)",42.101448,-83.160279
3,1,9009165600,"(41.3558996, -72.9323558)",41.355900,-72.932356
4,3,36061001200,"(40.7159065, -73.9820936)",40.715907,-73.982094


In [3]:
true_types = get_true_types(df, label)
# print(true_types[:30])


In [4]:
rtree = create_rtree(df)

In [ ]:
# lat_max = df['lat'].values.max()
# lat_min = df['lat'].values.min()
# lon_max = df['lon'].values.max()
# lon_min = df['lon'].values.min()

# mapit = folium.Map(location=[37.09, -95.71], zoom_start=5, tiles="Stamen Toner")

# for index, row in df.iterrows():
#     if row[label] == 1:
#         folium.CircleMarker( location=(row['lat'], row['lon']), color='#00FF00', fill_color='#00FF00', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )
#     elif row[label] == 0:
#         folium.CircleMarker( location=(row['lat'], row['lon']), color='#FF0000', fill_color='#FF0000', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )


# mapit.fit_bounds([(lat_min, lon_min), (lat_max, lon_max)])

# mapit

# 2. Run Experiments

There are three experiments:
- Unrestricted regions: runs **our approach** on unrestricted regions.
- One Partitioning: runs **our approach** against **MeanVar** on regions from a single partitioning.
- Multiple Partitionings: runs **MeanVar** on multiple partitionings.


## Unrestricted regions

In [5]:
seeds = create_seeds(df, rtree, 100)
# print(len(seeds), seeds[:10])

In [6]:
radii = np.arange(0.05, 1.01, 0.05)
regions = create_regions(df, rtree, seeds, radii)

print(len(regions), 'regions')


2000 regions


In [ ]:
mapit = folium.Map(location=[37.09, -95.71], zoom_start=5, tiles="Stamen Toner")

for point in seeds:
    folium.CircleMarker( location=id2loc(df, point), color='#0000FF', fill_color='#0000FF', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )

center = (26, -126)

r = radii[0]
folium.Rectangle([(center-r, center-r), (center+r, center+r)], color='#F1CF3B').add_to( mapit )
r = radii[-1]
folium.Rectangle([(center-r, center-r), (center+r, center+r)], color='#F1CF3B').add_to( mapit )


mapit


In [7]:
direction = 'both'
# direction = 'less_in'
# direction = 'less_out'

best_region, max_likeli, statistics = scan_regions(regions, true_types, N, P, direction=direction, verbose=True)

# statistics.sort(key=lambda x: -x)
# print(statistics)


range 4.656030796468258e-07 1963.0672387208906
max likelihood 1963.0672387208906
n=17874, p=14719
rho=0.6166419595190342, rho_in=0.8234866286225803, rho_out=0.5970330532926001
l0max=-137409.18985450186, l1max=-135446.12261578097, statistic=1963.0672387208906


In [8]:
## determine the significance threshold based on a desired signif_level

n_alt_worlds = 200
signif_level = 0.005

signif_thresh = get_signif_threshold(signif_level, n_alt_worlds, regions, N, P)
print(signif_thresh)

9.920751262805425


In [9]:
## identify regions with statistic above statistical significance threshold

sorted_statistics = np.sort(statistics)

top_k = len(statistics) - np.searchsorted(sorted_statistics, signif_thresh)

print(top_k, 'significant regions')


indexes = np.argsort(statistics)[::-1][:top_k]

significant_regions = [ regions[i] for i in indexes ]

695 significant regions


In [10]:
def intersects(regionA, regionB):
    cA = np.array(id2loc(df, regionA['center']))
    cB = np.array(id2loc(df, regionB['center']))
    rA = regionA['radius']
    rB = regionB['radius']

    A_top_right = cA + np.array([rA, rA])
    A_bottom_left = cA - np.array([rA, rA])
    B_top_right = cB + np.array([rB, rB])
    B_bottom_left = cB - np.array([rB, rB])

    # print(A_bottom_left, A_top_right, B_bottom_left, B_top_right)

    return not (A_top_right[0] < B_bottom_left[0] or A_bottom_left[0] > B_top_right[0] or A_top_right[1] < B_bottom_left[1] or A_bottom_left[1] > B_top_right[1])



non_olap_regions = []
centers = []
for region in significant_regions:
    center = region['center']
    if center in centers:
        continue
    
    no_intersections = True
    for other in non_olap_regions:
        if intersects(region, other):
            no_intersections = False
            break
    if no_intersections:
        centers.append(center)
        non_olap_regions.append(region)
    # print(region['radius'])

print(len(non_olap_regions), 'non-overlapping regions')

# over(non_olap_regions[0], non_olap_regions[20])

41 non-overlapping regions


In [11]:
## find smallest, largest regions

min_radius = np.inf
max_radius = -np.inf
for region in non_olap_regions:
    if region['radius'] < min_radius:
        min_radius = region['radius']
        # region_min_radius = region
    if region['radius'] > max_radius:
        max_radius = region['radius']
        # region_max_radius = region

min_points = np.inf
max_points = -np.inf
for region in non_olap_regions:
    if region['radius'] == min_radius and len(region['points']) < min_points:
        min_points = len(region['points'])
        region_min_radius = region
    if region['radius'] == max_radius and len(region['points']) > max_points:
        max_points = len(region['points'])
        region_max_radius = region

print(len(region_min_radius['points']), len(region_max_radius['points']))

600 7028


In [ ]:
show_circular_regions(df, true_types, non_olap_regions[:5])

# show_circular_regions(df, true_types, [region_min_radius, region_max_radius])

## One Partitioning

In [12]:
def create_partitioning(df, rtree, lon_min: float, lon_max: float, lat_min: float, lat_max: float, lon_n: float, lat_n: float):
    grid_info = {}
    grid_info['lon_min'] = lon_min
    grid_info['lon_max'] = lon_max
    grid_info['lat_min'] = lat_min
    grid_info['lat_max'] = lat_max
    grid_info['lat_n'] = lat_n
    grid_info['lon_n'] = lon_n

    grid_loc2_idx = {} ## maps (x,y) grid_loc coords to an index in the partitions array

    partitions = []
    for i in range(lat_n):
        lat_start = lat_min + (i/lat_n)*(lat_max - lat_min)
        lat_end = lat_min + ((i+1)/lat_n)*(lat_max - lat_min)
        for j in range(lon_n):
            lon_start = lon_min + (j/lon_n)*(lon_max - lon_min)
            lon_end = lon_min + ((j+1)/lon_n)*(lon_max - lon_min)

            points = query_range_box(df, rtree, lon_start, lon_end, lat_start, lat_end)
            # print(len(points))
            partition  = {
                'grid_loc': (j, i),
                'points' : points,
            }
            grid_loc2_idx[(j,i)] = len(partitions)
            partitions.append(partition)
    
    return grid_info, grid_loc2_idx, partitions


In [13]:
lat_max = df['lat'].values.max()
lat_min = df['lat'].values.min()
lon_max = df['lon'].values.max()
lon_min = df['lon'].values.min()
print(lat_min, lat_max, lon_min, lon_max)

19.5361762 64.8944045 -161.8508035 -67.1081422


In [14]:
### create the partitioning (grid) and its partitions (regions)

# lat_n = 12 ## number of partitions along vertical axis (latitude)  ## was 12
# lon_n = 25 ## number of partitions along horizontal axis (longitude) ## was 25

lat_n = 20
lon_n = 20


grid_info, grid_loc2_idx, regions = create_partitioning(df, rtree, lon_min, lon_max, lat_min, lat_max, lon_n, lat_n)

### Our Method

In [15]:
best_region, max_likeli, statistics = scan_regions(regions, true_types, N, P, verbose=True)


range 0.0 1313.1253486430214
max likelihood 1313.1253486430214
n=13656, p=11101
rho=0.6166419595190342, rho_in=0.8129027533684827, rho_out=0.6027380915325635
l0max=-137409.18985450186, l1max=-136096.06450585884, statistic=1313.1253486430214


In [16]:
## determine the significance threshold based on a desired signif_level

n_alt_worlds = 1000
signif_level = 0.005

signif_thresh = get_signif_threshold(signif_level, n_alt_worlds, regions, N, P)
print(signif_thresh)

7.900119686091784


In [17]:
## identify regions with statistic above statistical significance threshold

sorted_statistics = np.sort(statistics)
# print(sorted_statistics[::-1][40:60])
# print(np.sort(statistics)[::-1][40:60])

top_k = len(statistics) - np.searchsorted(sorted_statistics, signif_thresh)

print(top_k, 'significant regions')


indexes = np.argsort(statistics)[::-1][:top_k]

significant_regions = [ regions[i] for i in indexes ]


43 significant regions


In [ ]:
# show_grid_region(df, grid_info, true_types, best_region)
show_grid_regions(df, grid_info, true_types, significant_regions[:])


### MeanVar Method

In [18]:
## partioning-based scan

the_region, max_score, scores = scan_partitioning(regions, true_types)

print('max_score', max_score, 'with', len(the_region['points']), 'points')


## get the top_k regions

top_k = 5

ma = np.ma.masked_array(scores, mask=np.isnan(scores))

print(-np.sort(-ma)[:top_k])

indexes = np.argsort(-ma)[:top_k]

# print(indexes)

top_regions = [ regions[i] for i in indexes ]

max_score 0.3064977230247631 with 8 points
[0.3064977230247631 0.3064977230247631 0.3064977230247631
 0.3064977230247631 0.3064977230247631]


In [ ]:
# show_grid_region(df, grid_info, true_types, the_region)
show_grid_regions(df, grid_info, true_types, top_regions)


In [19]:
## best_region vs the_region

the_region = top_regions[0]

best_idx = grid_loc2_idx[best_region['grid_loc']]
the_idx = grid_loc2_idx[the_region['grid_loc']]

print(best_region['grid_loc'], the_region['grid_loc'])
print(best_idx, the_idx)

print(statistics[best_idx], statistics[the_idx])
print(scores[best_idx], scores[the_idx])


(8, 7) (7, 12)
148 247
1313.1253486430214 0.9587897912424523
0.06722631979411768 0.3064977230247631


## Multiple Partitionings

In [20]:
lat_max = df['lat'].values.max()
lat_min = df['lat'].values.min()
lon_max = df['lon'].values.max()
lon_min = df['lon'].values.min()
print(lat_min, lat_max, lon_min, lon_max)

19.5361762 64.8944045 -161.8508035 -67.1081422


In [21]:
lat_n_range = (10, 40)
lon_n_range = (10, 40)

n_partitionings = 100

partitionings = []

for i in range(n_partitionings):
    lat_n = random.randint(*lat_n_range)
    lon_n = random.randint(*lon_n_range)

    grid_info, grid_loc2_idx, regions = create_partitioning(df, rtree, lon_min, lon_max, lat_min, lat_max, lon_n, lat_n)

    partitionings.append((grid_info, grid_loc2_idx, regions))


### MeanVar Method

In [22]:
mean_scores = []
max_scores = []
for partitioning in partitionings:
    the_region, max_score, scores = scan_partitioning(partitioning[2], true_types)
    mean_score = np.nanmean(scores)
    mean_scores.append(mean_score)
    
    max_scores.append(max_score)

    # print(f'{mean_score=:.4f}, {max_score=:.4f}')

print(f'mean of means={np.mean(mean_scores):.4f}')
print(f'max of maxs={np.max(max_scores):.4f}')

mean of means=0.0376
max of maxs=0.3351
